# Создание датасета для обучения Clip

Импорты и подготовка

In [ ]:
import asyncio
import ast
import json
import os
from functools import partial
from pathlib import Path
from time import sleep
from typing import Iterator

import requests
import dashscope
from tqdm import tqdm
from tqdm.asyncio import tqdm_asyncio

os.chdir('..') # работаем в корне (clip_fine_tuning)

from dataset.src.repository import Clip993Repository


Класс ApiKeys (для перебора ключей и отключения перегруженных)

In [29]:
class ApiKeys(Iterator[str]):
    """
    Класс для управления списком API-ключей с поддержкой ротации и исключения невалидных ключей.
    """

    def __init__(self, items: list[str]) -> None:
        self.items: list[str] = items
        self.index: int = 0

    def __next__(self) -> str:
        value = self.items[self.index]
        self.index = (self.index + 1) % len(self.items)
        return value

    def __bool__(self) -> bool:
        return bool(self.items)

    def __len__(self) -> int:
        return len(self.items)

    def remove(self, item: str) -> None:
        if item not in self.items:
            return
        idx = self.items.index(item)
        self.items.remove(item)
        if not self.items:
            raise RuntimeError("Список API-ключей пуст")
        if idx < self.index:
            self.index -= 1
        self.index %= len(self.items)

    def __iter__(self) -> Iterator[str]:
        return self


api_keys = ApiKeys(json.load(open('dataset/qwen_api_keys.json'))) # Загрузка API-ключей

Загрузка изображений из [api picsum](https://picsum.photos/)

In [3]:
img_urls = []
for page in range(11):
    response = requests.get(f'https://picsum.photos/v2/list?page={page}&limit=100', timeout=15)
    if response.status_code == 200:
        img_urls += ['/'.join(image['download_url'].split('/')[:5]) + '/500/500' for image in response.json()]
    else:
        img_urls = []
        raise RuntimeError()
    sleep(5)

- Qwen не справляется с редиректами, ведущими на изображения, поэтому мы заранее разрешаем такие ссылки, сохраняем финальные URL в кэш, чтобы не перегружать API при повторных запросах.


In [4]:
CACHE_PATH = Path(".cache/img_hmac_urls.json")
CACHE_PATH.parent.mkdir(parents=True, exist_ok=True)
if CACHE_PATH.exists():
    with open(CACHE_PATH, "r") as f:
        img_hmac_urls = json.load(f)
else:
    img_hmac_urls = {}
for img_url in tqdm(img_urls):
    if img_url in img_hmac_urls:
        continue
    final_url = requests.get(img_url, timeout=10, verify=False).url
    img_hmac_urls[img_url] = final_url
    with open(CACHE_PATH, "w") as f:
        json.dump(img_hmac_urls, f, indent=2)
    sleep(10)

100%|██████████| 1093/1093 [00:00<00:00, 403447.53it/s]


- Промпт для генерации описаний

In [5]:
prompt = '''
Cоздай список из 10 коротких описаний (3–7 слов каждое) на русском языке,
описывающих это изображение. Используй формат Python-списка строк, например:
["Собака бежит по снегу", "Пёс играет на улице", ..., "Активная прогулка в зимнем лесу"].
'''

- Асинхронный вызов модели qwen

In [ ]:
semaphore = asyncio.Semaphore(len(api_keys))

async def get_label_of_image(img_url):
    async with semaphore:
        loop = asyncio.get_event_loop()
        current_key = next(api_keys)
        dashscope.api_key = current_key
        messages = [{
            'role': 'user',
            'content': [
                {'image': img_hmac_urls[img_url]},
                {'text': prompt},
            ]
        }]
        response = await loop.run_in_executor(
            None,
            partial(
                dashscope.MultiModalConversation.call,
                model='qwen2.5-vl-72b-instruct',
                messages=messages
            )
        )
        if response['status_code'] != 200:
            api_keys.remove(current_key)
            return
        label_text = response.output.choices[0].message.content[0]['text']
        Clip993Repository.add(img_url, ast.literal_eval(label_text))
        await asyncio.sleep(10)

Запуск асинхронных задач

In [ ]:
urls = list({url for url in img_urls if not Clip993Repository.get_by_url(url)})
while urls:
    tasks = [get_label_of_image(url) for url in urls]
    try:
        for coro in tqdm_asyncio.as_completed(tasks, desc="Processing images", total=len(tasks)):
            await coro
    except RuntimeError as e:
        print(f"RuntimeError: {e}")
    urls = list({url for url in img_urls if not Clip993Repository.get_by_url(url)})
    await asyncio.sleep(60)

*Все сохранено в `clip.db` (`clip993`)*